In [11]:
import ast

import pandas as pd
from owlready2 import *

In [16]:
onto = get_ontology("../../data/ontology_extended/cds-ontology_final-version_v5_new-iris.owl").load()

wv = Namespace(
    world_or_ontology=onto,
    base_iri="https://wissen-vernetzen.github.io/ontology.html#",
    name="wv",
)

In [17]:
df = pd.read_csv("../../data/filtered_cds_data.csv")

In [19]:
for letter in onto.Letter.instances():
    # ADDING SENTIMENT
    sentiment = df.loc[df['URL'] == letter.iri, 'sentiment'].item()

    if not sentiment == '0':
        for sentiment_instance in onto.Sentiment.instances():
            if sentiment_instance.iri.replace('https://wissen-vernetzen.github.io/ontology.html#', '') == sentiment:
                letter.has_sentiment.append(sentiment_instance)

    # ADDING KEYWORD
    keywords = df.loc[df['URL'] == letter.iri, 'keywords_full-letter'].item()
    if not keywords == '0':
        new_keywords = []
        for keyword in ast.literal_eval(keywords):
            keyword_id = ''.join(e for e in keyword if e.isalnum())
            if keyword_id not in onto.Automatic_Keyword.instances():
                new_keyword = onto.Automatic_Keyword(keyword_id, namespace=wv)
                new_keyword.label = keyword
                new_keywords.append(new_keyword)
            else:
                for keyword_instance in onto.Automatic_Keyword.instances():
                    if keyword_instance.iri.replace('https://wissen-vernetzen.github.io/ontology.html#', '') == keyword_id:
                        new_keywords.append(keyword_instance)

        for keyword in new_keywords:
            letter.has_automatic_keyword.append(keyword)

https://wissen-vernetzen.github.io/ontology.html#neutral
https://wissen-vernetzen.github.io/ontology.html#neutral
https://wissen-vernetzen.github.io/ontology.html#positive
https://wissen-vernetzen.github.io/ontology.html#negative
https://wissen-vernetzen.github.io/ontology.html#positive
https://wissen-vernetzen.github.io/ontology.html#neutral
https://wissen-vernetzen.github.io/ontology.html#neutral
https://wissen-vernetzen.github.io/ontology.html#positive
https://wissen-vernetzen.github.io/ontology.html#neutral


In [20]:
onto.save(file = "../../data/ontology_extended/why.owl", format = "rdfxml")